### Snowflake 데이터 및 Semantic Model YAML 파일 업로드 및 데이터 적재 가이드

이 단계에서는 Git 리포지토리에 포함된 데이터 파일과 YAML 파일을 Snowflake에 업로드하고, 미리 생성한 테이블에 데이터를 로드합니다.

---

#### 📌 준비된 파일 확인

다음 네 개의 파일을 확인하세요:

- `daily_revenue.csv`
- `region.csv`
- `product.csv`
- `revenue_timeseries.yaml`

> 위 파일들은 이전 단계에서 클론한 Git 리포지토리에 있습니다.

---

#### 1️⃣ 데이터 파일 업로드 (Stage로 로드)

Snowsight에서 데이터를 업로드합니다.

- **Snowsight**의 **Data 탭** → **Add Data** 클릭
- **Load files into a stage** 선택
- 위의 네 개의 파일 선택 후 업로드
- 데이터베이스와 스키마, 스테이지 설정:
    - Database: **`CORTEX_ANALYST_DEMO`**
    - Schema: **`REVENUE_TIMESERIES`**
    - Stage: **`RAW_DATA`**
- **Upload** 클릭하여 완료
![stage_업로드_완료](https://quickstarts.snowflake.com/guide/getting_started_with_cortex_analyst/img/c861a8372f9f287f.png)






---

### 2️⃣ 업로드한 파일 확인하기

Snowsight에서 업로드된 파일이 제대로 올라왔는지 확인합니다.

- **Data** → **Databases** 메뉴 선택
- 데이터베이스: **`CORTEX_ANALYST_DEMO`**, 스키마: **`REVENUE_TIMESERIES`** 선택
- **Stages** → **`RAW_DATA`** 스테이지 클릭
- 디렉터리 테이블이 비활성화 상태라면 **Enable Directory Table**을 활성화 후 웨어하우스 (**`CORTEX_ANALYST_WH`**) 선택
- 네 개의 파일(`daily_revenue.csv`, `region.csv`, `product.csv`, `revenue_timeseries.yaml`)이 리스트에 나타나는지 확인

---

### 3️⃣ CSV 데이터를 테이블로 적재하기

업로드한 CSV 파일을 미리 만들어진 테이블에 적재하려면, Snowflake SQL Worksheet에서 다음 쿼리를 실행하세요. (`load_data.sql`에서 제공된 예시입니다.)

In [ ]:
%%-- beginsql
/*--
• looad data into tables
--*/

USE ROLE CORTEX_USER_ROLE;
USE DATABASE CORTEX_ANALYST_DEMO;
USE SCHEMA CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES;
USE WAREHOUSE CORTEX_ANALYST_WH;

COPY INTO CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.DAILY_REVENUE
FROM @raw_data
FILES = ('daily_revenue.csv')
FILE_FORMAT = (
    TYPE=CSV,
    SKIP_HEADER=1,
    FIELD_DELIMITER=',',
    TRIM_SPACE=FALSE,
    FIELD_OPTIONALLY_ENCLOSED_BY=NONE,
    REPLACE_INVALID_CHARACTERS=TRUE,
    DATE_FORMAT=AUTO,
    TIME_FORMAT=AUTO,
    TIMESTAMP_FORMAT=AUTO
    EMPTY_FIELD_AS_NULL = FALSE
    error_on_column_count_mismatch=false
)

ON_ERROR=CONTINUE
FORCE = TRUE ;



COPY INTO CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.PRODUCT_DIM
FROM @raw_data
FILES = ('product.csv')
FILE_FORMAT = (
    TYPE=CSV,
    SKIP_HEADER=1,
    FIELD_DELIMITER=',',
    TRIM_SPACE=FALSE,
    FIELD_OPTIONALLY_ENCLOSED_BY=NONE,
    REPLACE_INVALID_CHARACTERS=TRUE,
    DATE_FORMAT=AUTO,
    TIME_FORMAT=AUTO,
    TIMESTAMP_FORMAT=AUTO
    EMPTY_FIELD_AS_NULL = FALSE
    error_on_column_count_mismatch=false
)

ON_ERROR=CONTINUE
FORCE = TRUE ;



COPY INTO CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.REGION_DIM
FROM @raw_data
FILES = ('region.csv')
FILE_FORMAT = (
    TYPE=CSV,
    SKIP_HEADER=1,
    FIELD_DELIMITER=',',
    TRIM_SPACE=FALSE,
    FIELD_OPTIONALLY_ENCLOSED_BY=NONE,
    REPLACE_INVALID_CHARACTERS=TRUE,
    DATE_FORMAT=AUTO,
    TIME_FORMAT=AUTO,
    TIMESTAMP_FORMAT=AUTO
    EMPTY_FIELD_AS_NULL = FALSE
    error_on_column_count_mismatch=false
)

ON_ERROR=CONTINUE
FORCE = TRUE ;
-- endsql

## Cortex Search 통합 가이드

Cortex Search를 통합하여 Cortex Analyst가 자연어 쿼리에서 정확한 문자열 검색 및 필터링을 통해 더욱 정확한 SQL을 생성하도록 지원합니다.

---

### 📌 Cortex Search란?

**Cortex Search**는 Snowflake에서 제공하는 기능으로, 사용자의 자연어 질문에서 필요한 **문자열(리터럴 값)**을 빠르고 정확하게 찾아내어 데이터 분석의 정확도를 높이는 검색 서비스입니다.

---

### ✅ Cortex Search가 필요한 이유

자연어 질의를 사용하다 보면 다음과 같은 상황이 생깁니다:

- 사용자는 정확한 데이터의 값을 항상 알지 못할 수 있습니다.
- 정확한 SQL 질의를 생성하려면 질문에서 사용되는 문자열을 데이터에서 정확히 찾아내는 것이 중요합니다.

**예시:**

> "**지난달 서울 지역의 매출은 얼마인가요?**"

여기서 "**서울**"이라는 값을 데이터에서 정확히 찾지 못하면, 올바른 SQL 쿼리를 만들기 어렵습니다.

---

### ⚙️ Cortex Search 작동 원리

Cortex Search는 아래와 같은 과정으로 작동합니다:

1. 사용자가 자연어로 질문 입력
2. Cortex Search가 미리 구축된 검색 인덱스에서 필요한 문자열을 검색
3. 찾은 문자열을 사용하여 Cortex Analyst가 정확한 SQL 쿼리 생성
4. 정확한 분석 결과를 사용자에게 전달


###


---

### 📝 활용 예시

| 사용자 질문                           | Cortex Search로 찾는 문자열 | 결과                                 |
|-----------------------------------|----------------------|------------------------------------|
| “서울 지역의 3월 매출은?”               | `서울`                | 정확한 지역 필터링이 가능해짐               |
| “스포츠 상품 라인의 매출은?”             | `스포츠`              | 특정 상품 라인을 정확하게 필터링 가능 |

---

### 📚 요약 및 핵심 정리

- Cortex Search는 자연어 질문에서 필요한 값을 찾는 **검색 인덱스 기능**입니다.
- SQL 질의의 정확도를 높여, **빠르고 정확한 데이터 분석**을 지원합니다.
- 데이터 분석의 **효율성과 정확성 향상**을 위한 필수 기능입니다.


### 🚩 Cortex Search 데이터 적재

Snowflake SQL Worksheet에서 다음의 `cortex_search_create.sql` 코드를 실행하여 Cortex Search를 위한 데이터를 적재합니다.

In [ ]:
%%-- beginsql
USE DATABASE cortex_analyst_demo;
USE SCHEMA revenue_timeseries;
use ROLE cortex_user_role;

  CREATE OR REPLACE CORTEX SEARCH SERVICE product_line_search_service
  ON product_dimension
  WAREHOUSE = cortex_analyst_wh
  TARGET_LAG = '1 hour'
  AS (
      SELECT DISTINCT product_line AS product_dimension FROM product_dim
  );
%%-- endsql